# Internet and Websites Search using Bing API - Bing Chat Clone

In this notebook, we'll delve into the ways in which you can **boost your GPT Smart Search Engine with web search functionalities**, utilizing both Langchain and the Azure Bing Search API service.

As previously discussed in our other notebooks, **harnessing agents and tools is an effective approach**. We aim to leverage the capabilities of OpenAI's large language models (LLM), such as GPT-4 and its successors, to perform the heavy lifting of reasoning and researching on our behalf.

There are numerous instances where it is necessary for our Smart Search Engine to have internet access. For instance, we may wish to **enrich an answer with information available on the web**, or **provide users with up-to-date and recent information**, or **finding information on an specific public website**. Regardless of the scenario, we require our engine to base its responses on search results.

By the conclusion of this notebook, you'll have a solid understanding of the Bing Search API basics, including **how to create a Web Search Agent using the Bing Search API**, and how these tools can strengthen your chatbot. Additionally, you'll learn about **Callback Handlers, their use, and their significance in bot applications**.

In [1]:
import requests
from typing import Dict, List
from pydantic import BaseModel, Extra, root_validator

from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager
from langchain.agents import initialize_agent, AgentType
from langchain.tools import BaseTool
from langchain.utilities import BingSearchAPIWrapper

from common.callbacks import StdOutCallbackHandler
from common.prompts import BING_PROMPT_PREFIX

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env")

def printmd(string):
    display(Markdown(string))
    
MODEL_DEPLOYMENT_NAME = "gpt-4-32k" # GPT-4 models are necessary for this feature. GPT-35-turbo will make mistakes on following system prompt instructions.

In [2]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

## Introduction to Callback Handlers

This following explanation comes directly from the Langchain documentation about Callbacks ([HERE](https://python.langchain.com/docs/modules/callbacks/)):

**Callbacks**:<br>
LangChain provides a callbacks system that allows you to hook into the various stages of your LLM application. This is useful for logging, monitoring, streaming, and other tasks. You can subscribe to these events by using the callbacks argument available throughout the API. This argument is list of handler objects.

**Callback handlers**:<br>
CallbackHandlers are objects that implement the CallbackHandler interface, which has a method for each event that can be subscribed to. The CallbackManager will call the appropriate method on each handler when the event is triggered.

--------------------
We will incorporate a handler for the callbacks, enabling us to observe the response as it streams and to gain insights into the Agent's reasoning process. This will prove incredibly valuable when we aim to stream the bot's responses to users and keep them informed about the ongoing process as they await the answer.

Our custom handler is on the folder `common/callbacks.py`. Go and take a look at it.

In [3]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

# Now we declare our LLM object with the callback handler 
llm = AzureChatOpenAI(deployment_name=MODEL_DEPLOYMENT_NAME, temperature=0, max_tokens=1000)

## Creating a custom tool - Bing Search API tool

Langhain has already a pre-created tool called BingSearchAPIWrapper ([HERE](https://github.com/hwchase17/langchain/blob/master/langchain/utilities/bing_search.py)), however we are going to make it a bit better by using the results function instead of the run function, that way we not only have the text results, but also the title and link(source) of each snippet.

In [4]:
class MyBingSearch(BaseTool):
    """Tool for a Bing Search Wrapper"""
    
    name = "@bing"
    description = "useful when the questions includes the term: @bing.\n"

    k: int = 5
    
    def _run(self, query: str) -> str:
        bing = BingSearchAPIWrapper(k=self.k)
        return bing.results(query,num_results=self.k)
            
    async def _arun(self, query: str) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("ChatGPTTool does not support async")

Now, we create our REACT agent that uses our custom tool and our custom prompt `BING_PROMPT_PREFIX`

In [5]:
tools = [MyBingSearch(k=5)]
agent_executor = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         agent_kwargs={'prefix':BING_PROMPT_PREFIX}, callback_manager=cb_manager, )

Try some of the below questions, or others that you might like

In [31]:
# QUESTION = "What is happening with the oil supply in the world right now?"
# QUESTION = "How much is 50 USD in Euros and is it enough for an average hotel in Madrid?"
# QUESTION = "My son needs to build a pinewood car for a pinewood derbi, how do I build such a car?"
# QUESTION = "Who won the 2023 superbowl and who was the MVP?"
QUESTION = "can I travel to Hawaii, Maui from Dallas, TX for 7 days with $7000 on the month of September, what are the best days to travel?"

# This complex question below needs gpt-4-32k (0613 version) in order to ensure a good answer. 
QUESTION = """
compare the number of job opennings (provide the exact number), the average salary within 15 miles of Dallas, TX, for these ocupations:

- ADN Registerd Nurse 
- Occupational therapist assistant
- Dental Hygienist
- Graphic Designer
- Real Estate Agent


Create a table with your findings. Place the sources on each cell.
Substitute `$` for `USD` in your final answer.
"""

QUESTION = """
Find appartments for rent in Las Colinas, TX under $1,000 per month, 1 bedroom and 1 bathroom, provide the location and links in a table.
Substitute `$` for `USD` in your final answer.
"""

In [32]:
#As LLMs responses are never the same, we do a for loop in case the answer cannot be parsed according to our prompt instructions
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking for a specific search for apartments for rent in Las Colinas, TX with a budget of under $1,000 per month, 1 bedroom and 1 bathroom. I will need to perform a web search to find this information. 

Action: @bing
Action Input: Apartments for rent in Las Colinas, TX under USD 1,000 per month, 1 bedroom and 1 bathroom
The search results provide several websites where I can find apartments for rent in Las Colinas, TX under USD 1,000 per month, with 1 bedroom and 1 bathroom. I will now perform a search on the first two websites from the search results to find specific apartments that meet the user's criteria.

Action: @bing
Action Input: site:https://www.zillow.com/ Apartments for rent in Las Colinas, TX under USD 1,000 per month, 1 bedroom and 1 bathroom
The search on Zillow did not provide specific apartments that meet the user's criteria. I will now perform a search on the second website from the initial search results.

Action: @bing
Action Input: site:https://www.apart

In [33]:
printmd(response)

Here is the apartment that fits your criteria:

| Apartment Name | Address | Rent Range (USD) | Number of Bedrooms | Link |
| --- | --- | --- | --- | --- |
| Water Ridge | 4600 W Pioneer Dr, Irving, TX 75061 | 890 - 1,765 | 1-3 | [Water Ridge](https://www.apartments.com/irving-tx/under-1000/) |

Please note that the rent range and number of bedrooms are for the entire property, so you may want to contact them for more specific information about 1 bedroom apartments under USD 1,000.

## QnA to specific websites

There are several use cases where we want the smart bot to answer questions about a specific company's public website. There are two approaches we can take:

1- Create a crawler script that runs regularly, finds every page on the website, and pushes the documents to Azure Cognitive Search.
2- Since Bing has likely already indexed the public website, we can utilize Bing search targeted specifically to that site, rather than attempting to index the site ourselves and duplicate the work already done by Bing's crawler.

Below are some sample questions related to specific sites. Take a look:

In [42]:
# QUESTION = """
# information on how to kill wasps in homedepot.com
# """

QUESTION = """
in target.com, find how what's the price of a Nesspresso coffee machine and of a Keurig coffee machine
"""

# QUESTION = """
# in microsoft.com, find out what is the latests news on quantum computing
# """

In [43]:
for i in range(3):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

The user is asking for the prices of a Nespresso coffee machine and a Keurig coffee machine on the website target.com. I will need to perform two separate searches to find the prices of each item.

Action: @bing
Action Input: site:target.com Nespresso coffee machine price
The search results indicate that the price of a Nespresso VertuoPlus Coffee Maker and Espresso Machine by DeLonghi is USD 199.99 on target.com<sup><a href="https://www.target.com/c/coffee-makers-tea-espresso-kitchen-appliances-dining/nespresso/-/N-5xtroZ5s79h">[1]</a></sup>. Now I will search for the price of a Keurig coffee machine on the same website.

Action: @bing
Action Input: site:target.com Keurig coffee machine price


In [44]:
printmd(response)

The price of a Nespresso VertuoPlus Coffee Maker and Espresso Machine by DeLonghi on target.com is USD 199.99<sup><a href="https://www.target.com/c/coffee-makers-tea-espresso-kitchen-appliances-dining/nespresso/-/N-5xtroZ5s79h">[1]</a></sup>, and the price of a Keurig coffee maker is USD 139.99<sup><a href="https://www.target.com/c/coffee-makers-tea-espresso-kitchen-appliances-dining/keurig/-/N-5xtroZ5uvm4">[2]</a></sup>. Is there anything else you would like to know?

In [45]:
# Uncomment if you want to take a look at the custom bing search prompt
# printmd(agent_executor.agent.llm_chain.prompt.template)

# Summary

In this notebook, we learned about Callback Handlers and how to stream the response from the LLM. We also learn how to create a Bing Chat clone using a clever prompt with specific search and formatting instructions.

The result is an agent that can smartly search the web for us and give us the answer to our question with the right url citations and links!

# NEXT

The Next Notebook will guide you on how we stick everything together. How do we use the features of all notebooks and create a brain agent that can respond to any request accordingly.